In [103]:
import requests
import json
import pandas as pd

# reference: 
#### https://sammi-baba.hatenablog.com/entry/2018/12/25/074017  
#### http://www.ie110704.net/2016/11/11/e-stat-api%E3%82%92python%E3%81%8B%E3%82%89%E4%BD%BF%E3%81%A3%E3%81%A6%E5%9C%B0%E7%90%86%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F/
#### https://medium.com/@rode89/pandas%E8%A6%9A%E6%9B%B8-52252df4c231

In [104]:
with open("api_key.json", "r") as f:
    appid = f.read()

In [105]:
appid = appid.replace("\n", "")

In [106]:
appid

'30fb7abec39a136d4c54d1ec77a593c09221603f'

In [107]:
#appid = "###################################"
api_version = "2.1"
base_url = "http://api.e-stat.go.jp/rest/{api_version}/app/json/".format(api_version=api_version)



#get_type="getSimpleStatsData"
get_type="getStatsData"
#get_type="refDataset"
stats_data_id="0003215844"
#cd_cat_01="0000" # 国籍->全て
#cd_cat_02="0000" # 性別->男女
#cd_cat_03="00710" # 集計地域->全域
lv_area="2" # 集計レベル->都道府県レベル
section_header_flg="2" # セクションヘッダー->無し



url = base_url + "{get_type}?\
appId={appid}&\
statsDataId={stats_data_id}&\
lvArea={lv_area}&\
sectionHeaderFlg={section_header_flg}".format(
        get_type=get_type,
        appid=appid,
        stats_data_id=stats_data_id,
        #cd_cat_01=cd_cat_01,
        #cd_cat_02=cd_cat_02,
        #cd_cat_03=cd_cat_03,
        lv_area=lv_area,
        section_header_flg=section_header_flg
        )

# cdCat01={cd_cat_01}&cdCat02={cd_cat_02}&cdCat03={cd_cat_03}&\

In [108]:
#url = "http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?appId={}&lang=J&statsDataId=0003215844&metaGetFlg=N&cntGetFlg=N&sectionHeaderFlg=2".format(api_key)
res = requests.get(url)

In [109]:
res

<Response [200]>

In [110]:
res = res.json()

In [111]:
#res2 = json.loads(res.text)
#assert res1 == res2

In [112]:
def simple_parser(dic, tag):
    res = []
    for k,v in dic.items():
        if k == tag:
            res.append(v)
            return res
        elif isinstance(v, dict):
            for r in simple_parser(v, tag):
                res.append(r)
        else:
            []
    return res

In [113]:
values = simple_parser(res, "VALUE")[0] # list of dict_type objects

In [114]:
values[:2]

[{'@tab': '001',
  '@cat01': '000',
  '@cat02': '001',
  '@area': '01000',
  '@time': '2005000000',
  '@unit': '千人',
  '$': '5628'},
 {'@tab': '001',
  '@cat01': '000',
  '@cat02': '001',
  '@area': '01000',
  '@time': '2010000000',
  '@unit': '千人',
  '$': '5506'}]

In [115]:
class_obj = simple_parser(res, "CLASS_OBJ")[0]

In [116]:
class_obj

[{'@id': 'tab',
  '@name': '表章項目',
  'CLASS': {'@code': '001', '@name': '人口', '@level': '', '@unit': '千人'}},
 {'@id': 'cat01',
  '@name': '男女別',
  'CLASS': [{'@code': '000', '@name': '男女計', '@level': '1'},
   {'@code': '001', '@name': '男', '@level': '1'},
   {'@code': '002', '@name': '女', '@level': '1'}]},
 {'@id': 'cat02',
  '@name': '人口',
  'CLASS': [{'@code': '001', '@name': '総人口', '@level': '1'},
   {'@code': '002', '@name': '日本人人口', '@level': '1'}]},
 {'@id': 'area',
  '@name': '全国・都道府県',
  'CLASS': [{'@code': '01000', '@name': '北海道', '@level': '2'},
   {'@code': '02000', '@name': '青森県', '@level': '2'},
   {'@code': '03000', '@name': '岩手県', '@level': '2'},
   {'@code': '04000', '@name': '宮城県', '@level': '2'},
   {'@code': '05000', '@name': '秋田県', '@level': '2'},
   {'@code': '06000', '@name': '山形県', '@level': '2'},
   {'@code': '07000', '@name': '福島県', '@level': '2'},
   {'@code': '08000', '@name': '茨城県', '@level': '2'},
   {'@code': '09000', '@name': '栃木県', '@level': '2'},
   {'@

In [117]:
cat01 = class_obj[1]["@name"] #column name
cat01

'男女別'

In [118]:
cat02 = class_obj[2]["@name"] #column name
cat02

'人口'

In [119]:
cat03 = class_obj[3]["@name"]
cat03

'全国・都道府県'

In [120]:
cat04 = class_obj[4]["@name"]
cat04

'時間軸（年）'

In [121]:
col_info = {"@cat01":cat01, "@cat02":cat02, "@area":cat03, "@time":cat04 ,"$":"千人"}

cat01_map = {d["@code"]:d["@name"] for d in class_obj[1]["CLASS"]}
cat02_map = {d["@code"]:d["@name"] for d in class_obj[2]["CLASS"]}
cat03_map = {d["@code"]:d["@name"] for d in class_obj[3]["CLASS"]}
cat04_map = {d["@code"]:d["@name"] for d in class_obj[4]["CLASS"]}

df = pd.DataFrame(values, columns=["@cat01", "@cat02", "@area", "@time", "$"]).rename(columns=col_info)

df[cat01] = df[cat01].map(cat01_map)
df[cat02] = df[cat02].map(cat02_map)
df[cat03] = df[cat03].map(cat03_map)
df[cat04] = df[cat04].map(cat04_map)

In [122]:
df = df[(df["男女別"]=="男女計")&(df["人口"]=="総人口")]

In [123]:
df = df[["時間軸（年）", "全国・都道府県", "千人"]]

In [124]:
df["千人"] = df["千人"].apply(lambda x: int(x)*1000)

In [125]:
df.rename(columns={"千人":"人口", "時間軸（年）":"年"}, inplace=True)

In [126]:
df.head(20)

,年,全国・都道府県,人口
0,平成17年,北海道,5628000
1,平成22年,北海道,5506000
2,平成27年,北海道,5382000
3,平成28年,北海道,5352000
4,平成29年,北海道,5320000
5,平成17年,青森県,1437000
6,平成22年,青森県,1373000
7,平成27年,青森県,1308000
8,平成28年,青森県,1293000
9,平成29年,青森県,1278000
